# Debate a base
#### Update script (Parlamint 4.1)
- Counts the total amount of words spoken per country per day
- Outputted file needs to replace `static/helpers/proper_word_counts.csv` within the website (when you are working with a version which is NOT Parlamint 4.1)
- Just make sure all imports are working and you should be able to just run it

In [1]:
# Imports
import os

import warnings
warnings.filterwarnings('ignore')

import time
import re
import pandas as pd
from bs4 import BeautifulSoup as bs

In [2]:
# dir
translated_csv_dir = "../data/preprocessed"

In [36]:
# return list met dict [{Land1}, {Land 1}]
def get_csv_files(country_selection):
    # os.chdir(translated_csv_dir)
    
    country_return_list = []

    # loop door alle folders die hierboven zijn geprint
    for country in os.listdir(translated_csv_dir):
        
        # filter op specifiek land (IN BOX 2)
        if country == country_selection:
            paths_dict = {}

            # ga door alle inhoud van de landfolder heen
            for root, dirs, files in os.walk(os.path.join(translated_csv_dir, country)):
                file_data = []
                
                # loop door files van een folder
                for file in files:
                    
                    file_data.append(file)

                paths_dict[root.split("\\")[1]] = file_data

            country_return_list.append(paths_dict)
        
    return country_return_list

In [4]:
# haal jaar maand en dag uit filename
def extract_file_date(file_name):
    
    year_month_day = re.search(r"\d{4}-\d{2}-\d{2}", file_name)
#     year, month, day = year_month_day[0].split("-")
    
    return year_month_day[0]

In [29]:
# geeft een dict met dates als keys en files als vals
def get_files_per_date(country, after=None):
    files = get_csv_files(country)[0][country]
    
    dates_dict = {}
    
    if after is not None:
        
        after = time.mktime(datetime.datetime.strptime(after, "%Y-%m-%d").timetuple())
    
    # loop door alle files heen
    for file in files:
        
        date = extract_file_date(file)
        
        # filter al geuploade dates uit de dict
        if after is not None:
            timestamp = time.mktime(datetime.datetime.strptime(date, "%Y-%m-%d").timetuple())
            
            if (timestamp - after) <= 0:
                
                continue
        
        if date in dates_dict.keys():
            
            dates_dict[date] += [file]
            
        else:
            
            dates_dict[date] = [file]
            
    return dates_dict

In [12]:
# maakt paths aan voor alle files die bij een date horen
def get_date_paths(root, files):
    # os.chdir(translated_csv_dir)
    
    paths = []
    
    for file in files:
        paths.append(os.path.join(translated_csv_dir, root, file))
        
    return paths

In [7]:
# kijk hoeveel woorden in alle files van een date staan
def get_date_word_count(paths):
    word_count = 0
    
    for path in paths:
        df = pd.read_csv(path)

        for sentence in df["value"]:
            
            if type(sentence) != float:

                # haal special characters weg en haal vervolgens '' uit list weg
                sentence_list = list(filter(lambda x: len(x) > 0, sentence.translate ({ord(c): " " for c in '!@#$%^&*()[]{};:,./<>?\|`~-–=_+"'}).split(" ")))
                
                word_count += len(sentence_list)
                                     
            else:
                # add one word for each empty line in a file
                word_count += 1
        
    return word_count

### Progress
- The cell below should show a dataframe, after the entire notebook is finished

In [ ]:
# pak alle word counts van alle translated countries op datum
translated_countries = os.listdir(translated_csv_dir)
savethis_df = pd.DataFrame({'country': [],
                            'date': [],
                            'words': []})

for country in translated_countries:
    
    # loop door elke date heen en de files die er bij horen
    for date, files in get_files_per_date(country).items():
        
        # vraag per date de paths op van files
        count = get_date_word_count(get_date_paths(country, files))

        # zet alles in een df
        savethis_df.loc[len(savethis_df.index)] = [country, date, count]

display(savethis_df)

csv_loc = '../data/xml_word_counts.csv'

savethis_df.to_csv(csv_loc, index=False)

df_words_import = pd.read_csv(csv_loc)

display(df_words_import)

In [9]:
# # functie testruimte
# t_folder = "BE"
# t_date = '2014-07-30'

# test_dates_dict = get_files_per_date(t_folder)
# # print(test_dates_dict)
# test_paths = get_date_paths(t_folder, test_dates_dict[t_date ])
# # print(test_paths)
# test_count = get_date_word_count(test_paths)
# print(test_count)